In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
system_name = "siris-D-500kW"
location = "Greece"
base_url = "https://pvoutput.org/list.jsp?p={}&id=84471&sid=90854&gs=0&v=0&o=date&d=desc"
pages_to_scrape = 5

In [3]:
options = Options()
options.binary_location = r"C:\Program Files\chrome for testing\chrome-win64\chrome.exe"
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
service = Service(executable_path=r"C:\Program Files\chrome for testing\chrome-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [4]:
all_data = []

for page in range(pages_to_scrape):
    url = base_url.format(page)
    print(f"📄 Scraping page {page + 1} from: {url}")
    
    try:
        driver.get(url)
        time.sleep(6)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table", id="tbl_main")

        if not table:
            print(f"⚠️ No table found on page {page + 1}. Retrying once...")
            time.sleep(5)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            table = soup.find("table", id="tbl_main")

        if table:
            rows = table.find_all("tr")[1:]
            for row in rows:
                cols = [td.get_text(strip=True).replace(',', '') for td in row.find_all("td")]
                if len(cols) >= 9:
                    cols.append(system_name)
                    cols.append(location)
                    all_data.append(cols)
        else:
            print(f"❌ Failed again: No table found on page {page + 1}")
    except Exception as e:
        print(f"🚨 Error scraping page {page + 1}: {e}")

driver.quit()

📄 Scraping page 1 from: https://pvoutput.org/list.jsp?p=0&id=84471&sid=90854&gs=0&v=0&o=date&d=desc
📄 Scraping page 2 from: https://pvoutput.org/list.jsp?p=1&id=84471&sid=90854&gs=0&v=0&o=date&d=desc
📄 Scraping page 3 from: https://pvoutput.org/list.jsp?p=2&id=84471&sid=90854&gs=0&v=0&o=date&d=desc
📄 Scraping page 4 from: https://pvoutput.org/list.jsp?p=3&id=84471&sid=90854&gs=0&v=0&o=date&d=desc
📄 Scraping page 5 from: https://pvoutput.org/list.jsp?p=4&id=84471&sid=90854&gs=0&v=0&o=date&d=desc


In [5]:
header = [
    "Date", "Generated", "Efficiency", "Exported", "Peak Power",
    "Peak Time", "Conditions", "Temperature", "Comments",
    "System Name", "Location"
]

In [24]:
df = pd.DataFrame(all_data, columns=header)
df.drop(columns=["Exported", "Temperature", "Comments", "Peak Power", "Peak Time", "Conditions"], inplace=True)
print(df.head())

       Date    Generated   Efficiency    System Name Location
0  31/03/25     0.000kWh  0.000kWh/kW  siris-D-500kW   Greece
1  30/03/25  1681.506kWh  3.316kWh/kW  siris-D-500kW   Greece
2  29/03/25   598.146kWh  1.180kWh/kW  siris-D-500kW   Greece
3  28/03/25  1775.921kWh  3.502kWh/kW  siris-D-500kW   Greece
4  27/03/25  1866.433kWh  3.681kWh/kW  siris-D-500kW   Greece


In [26]:
page6 = pd.read_csv("siris-D_page6.csv")
page7 = pd.read_csv("siris-D_page7.csv")
page8 = pd.read_csv("siris-D_page8.csv")
page9 = pd.read_csv("siris-D_page9.csv")
page10 = pd.read_csv("siris-D_page10.csv")

In [28]:
page6["System Name"] = "siris-D-500kW"
page6["Location"] = "Greece"
page7["System Name"] = "siris-D-500kW"
page7["Location"] = "Greece"
page8["System Name"] = "siris-D-500kW"
page8["Location"] = "Greece"
page9["System Name"] = "siris-D-500kW"
page9["Location"] = "Greece"
page10["System Name"] = "siris-D-500kW"
page10["Location"] = "Greece"

In [30]:
master_df = pd.concat([df, page6, page7, page8, page9, page10], ignore_index=True)
master_df

Date    Generated   Efficiency    System Name Location  Unnamed: 2
0     31/03/25     0.000kWh  0.000kWh/kW  siris-D-500kW   Greece         NaN
1     30/03/25  1681.506kWh  3.316kWh/kW  siris-D-500kW   Greece         NaN
2     29/03/25   598.146kWh  1.180kWh/kW  siris-D-500kW   Greece         NaN
3     28/03/25  1775.921kWh  3.502kWh/kW  siris-D-500kW   Greece         NaN
4     27/03/25  1866.433kWh  3.681kWh/kW  siris-D-500kW   Greece         NaN
..         ...          ...          ...            ...      ...         ...
395  2/27/2024  2627.272kWh     5.180974  siris-D-500kW   Greece         NaN
396  2/26/2024   787.898kWh     1.553733  siris-D-500kW   Greece         NaN
397  2/25/2024  1006.772kWh     1.985352  siris-D-500kW   Greece         NaN
398  2/24/2024  2564.958kWh     5.058091  siris-D-500kW   Greece         NaN
399  2/23/2024  2101.483kWh      4.14412  siris-D-500kW   Greece         NaN

[400 rows x 6 columns]

In [32]:
master_df["System Size (kW)"] = 507.100

In [34]:
master_df.to_csv("daily_siris-D_master.csv", index=False)